In [35]:
import pandas as pd
import csv

In [36]:
csvPath = "/Users/janek/Documents/CREATIVE ZONE/wydatkiApp/wydatkiApp/rawData.csv"
MAIN_DF = pd.read_csv(csvPath)
df_wydatki = MAIN_DF

In [ ]:
df_wydatki.drop(df_wydatki.index[:18], inplace=True)
df_wydatki

In [53]:
import csv
import re
rawDataPath = "/Users/janek/Documents/CREATIVE ZONE/wydatkiApp/wydatkiApp/rawData.csv"

with open(rawDataPath, 'r') as file:
    content = file.read()

# Usunięcie wszystkiego, co pasuje do ((?s)^.*?)
cleaned_content = re.sub(r'(?s)^.*?Data transakcji', '', content)

#Nadaje ściekę dla nowego pliku
cleanedDataPath = 'cleanedData.csv'

#Otwiera plik cleanedData.csv do zapisu
with open(cleanedDataPath, 'w') as file:
    writer = csv.writer(file)
    # Write data to the CSV file
    writer.writerows(cleaned_content)

with open(cleanedDataPath, 'r') as file:
    content = file.read()
   
